# Project 08. Song Challenge

In [2]:
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import normalize


# Table of Contents
  1. [Read Data](#readdata)

<a id='read_data'></a>
## Read Data

In [3]:
data = pd.read_json('C:/Users/xiaoj/Desktop/CS/collection of data science-take home challenges/Data/8. song.json')
data.head()

,id,user_id,user_state,user_sign_up_date,song_played,time_played
0,GOQMMKSQQH,122,Louisiana,2015-05-16,Hey Jude,2015-06-11 21:51:35
1,HWKKBQKNWI,3,Ohio,2015-05-01,We Can Work It Out,2015-06-06 16:49:19
2,DKQSXVNJDH,35,New Jersey,2015-05-04,Back In the U.S.S.R.,2015-06-14 02:11:29
3,HLHRIDQTUW,126,Illinois,2015-05-16,P.s. I Love You,2015-06-08 12:26:10
4,SUKJCSBCYW,6,New Jersey,2015-05-01,Sgt. Pepper's Lonely Hearts Club Band,2015-06-28 14:57:00


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 4000 non-null   object
 1   user_id            4000 non-null   int64 
 2   user_state         4000 non-null   object
 3   user_sign_up_date  4000 non-null   object
 4   song_played        4000 non-null   object
 5   time_played        4000 non-null   object
dtypes: int64(1), object(5)
memory usage: 187.6+ KB


In [5]:
# transfer two time-related field from 'object' to time
data['user_sign_up_date'] = pd.to_datetime(data['user_sign_up_date'])
data['time_played'] = pd.to_datetime(data['time_played'])

In [6]:
# check if there are any null or NaN values
data.isnull().sum()

id                   0
user_id              0
user_state           0
user_sign_up_date    0
song_played          0
time_played          0
dtype: int64

In [9]:
# check the unique values
columns=['song_played','user_id','user_state']
[data[column].nunique() for column in columns]

[97, 196, 41]

<a id='Question1'></a>
## Question 1
What are the top 3 and the bottom 3 states in terms of number of users?

In [36]:
# sort the state according to their corresponding user counts
state_user_count = data.groupby('user_state')['user_id'].agg(lambda id:len(np.unique(id)))
state_user_count.sort_values(inplace=True, ascending = False)
state_user_count

user_state
New York          23
California        21
Texas             15
Pennsylvania       9
Ohio               9
Florida            7
Illinois           7
Georgia            6
New Jersey         6
North Carolina     6
Massachusetts      6
Michigan           5
Wisconsin          5
Maryland           5
Louisiana          5
Missouri           5
Tennessee          5
Indiana            4
Minnesota          4
Alabama            4
West Virginia      3
Oregon             3
Colorado           3
Kentucky           3
South Carolina     3
Mississippi        3
Oklahoma           2
Utah               2
Virginia           2
Washington         2
Arkansas           2
Alaska             2
North Dakota       1
Kansas             1
Iowa               1
Rhode Island       1
Nebraska           1
Idaho              1
Connecticut        1
New Mexico         1
Arizona            1
Name: user_id, dtype: int64

In [29]:
# Top 3 state 
state_user_count.iloc[:3]

user_state
New York      23
California    21
Texas         15
Name: user_id, dtype: int64

In [37]:
# Bottom 3 states, since the last 9 states has the same user counts, all of them have been shown here
state_user_count.iloc[-9:]

user_state
North Dakota    1
Kansas          1
Iowa            1
Rhode Island    1
Nebraska        1
Idaho           1
Connecticut     1
New Mexico      1
Arizona         1
Name: user_id, dtype: int64

<a id='question2'></a>
## Question 2